In [2]:
import pandas as pd

In [3]:
df_student_vle = pd.read_csv("data/studentVle.csv")
df_student_info = pd.read_csv("data/studentInfo.csv")
df_course = pd.read_csv("data/courses.csv")

In [4]:
print(df_student_vle.keys())
print(df_student_info.keys())
print(df_course.keys())

Index(['code_module', 'code_presentation', 'id_student', 'id_site', 'date',
       'sum_click'],
      dtype='object')
Index(['code_module', 'code_presentation', 'id_student', 'gender', 'region',
       'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts',
       'studied_credits', 'disability', 'final_result'],
      dtype='object')
Index(['code_module', 'code_presentation', 'module_presentation_length'], dtype='object')


On nettoie les données en retirant les étudiants qui ont abandonné

In [5]:
df_clean_withdrawn = df_student_info[df_student_info["final_result"] != "Withdrawn"]

In [6]:
max_length =df_course['module_presentation_length'].max()

df_student_clean = df_student_vle[(df_student_vle["date"] >= 0) & (df_student_vle["date"] <= max_length)]

In [7]:
df_clean_withdrawn = df_clean_withdrawn[["code_module", "code_presentation" ,"id_student", "final_result"]]

In [8]:
df_student_note = df_student_clean.merge(df_clean_withdrawn, left_on=["code_module", "code_presentation" ,"id_student"], right_on=["code_module", "code_presentation" ,"id_student"])

In [9]:
df_final = df_student_note.merge(df_course, right_on=["code_module", "code_presentation"], left_on=["code_module", "code_presentation"])

In [13]:
df_final["time_to_learn"] = df_final["module_presentation_length"] - df_final["date"]

In [15]:
binrange = range(0, max_length+2, 30)
labrange = range(0, int(max_length/30)+1)

In [16]:
df_final["rank"] = pd.cut(df_final['time_to_learn'],bins=binrange, labels=labrange)

In [18]:
df_final["id_student"].astype(str)

0          345357
1          345357
2          345357
3          345357
4          345357
            ...  
9144181    624767
9144182    624767
9144183    624767
9144184    624767
9144185    624767
Name: id_student, Length: 9144186, dtype: object

In [70]:
df_group = df_final.groupby(by=["rank","code_module","code_presentation", "id_student"], ).sum()

/tmp/ipykernel_369/1556827158.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_group = df_final.groupby(by=["rank","code_module","code_presentation", "id_student"]).sum()


In [72]:
df_groupe_reindex = df_group.reset_index()

In [74]:
df_groupe_reindex[df_groupe_reindex["id_student"] == 6516]

,rank,code_module,code_presentation,id_student,id_site,date,sum_click,module_presentation_length,time_to_learn
0,0,AAA,2013B,6516,0,0,0,0,0
20672,0,AAA,2013J,6516,0,0,0,0,0
41344,0,AAA,2014B,6516,0,0,0,0,0
62016,0,AAA,2014J,6516,4385224,1229,9,1345,116
82688,0,BBB,2013B,6516,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
5105984,8,FFF,2014J,6516,0,0,0,0,0
5126656,8,GGG,2013B,6516,0,0,0,0,0
5147328,8,GGG,2013J,6516,0,0,0,0,0
5168000,8,GGG,2014B,6516,0,0,0,0,0


In [56]:
df_groupe_reindex = df_groupe_reindex[["rank", "code_module", "code_presentation", "id_student", "sum_click"]]

In [57]:
df_groupe_reindex

,rank,code_module,code_presentation,id_student,sum_click
0,0,AAA,2013B,6516,0
1,0,AAA,2013B,11391,0
2,0,AAA,2013B,23629,0
3,0,AAA,2013B,23698,0
4,0,AAA,2013B,23798,0
...,...,...,...,...,...
5209339,8,GGG,2014J,2698251,0
5209340,8,GGG,2014J,2698257,0
5209341,8,GGG,2014J,2698535,0
5209342,8,GGG,2014J,2698577,0


In [59]:
df_pivot = df_groupe_reindex.pivot_table(index=["code_module", "code_presentation", "id_student"], columns="rank")

In [62]:
df_pivot = df_pivot.reset_index()

In [87]:
df_pivot

code_module code_presentation id_student sum_click                    \
rank                                                    0  1  2  3  4  5  6   
0              AAA             2013B       6516         0  0  0  0  0  0  0   
1              AAA             2013B      11391         0  0  0  0  0  0  0   
2              AAA             2013B      23629         0  0  0  0  0  0  0   
3              AAA             2013B      23698         0  0  0  0  0  0  0   
4              AAA             2013B      23798         0  0  0  0  0  0  0   
...            ...               ...        ...       ... .. .. .. .. .. ..   
578811         GGG             2014J    2698251         0  0  0  0  0  0  0   
578812         GGG             2014J    2698257         0  0  0  0  0  0  0   
578813         GGG             2014J    2698535         0  0  0  0  0  0  0   
578814         GGG             2014J    2698577         0  0  0  0  0  0  0   
578815         GGG             2014J    2698588         0  0  0  0  0  0  0   

              
rank    7  8  
0       0  0  
1       0  0  
2       0  0  
3       0  0  
4       0  0  
...    .. ..  
578811  0  0  
578812  0  0  
578813  0  0  
578814  0  0  
578815  0  0  

[578816 rows x 12 columns]

In [79]:
df_student_restrict = df_student_info[["code_module", "code_presentation", "id_student"]]

In [92]:
df_rest = df_pivot.merge(df_student_restrict, on=["code_module", "code_presentation", "id_student"])

/tmp/ipykernel_369/3846649955.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  df_rest = df_pivot.merge(df_student_restrict, on=["code_module", "code_presentation", "id_student"])


In [102]:
df_rest.columns =  df_rest.columns.get_level_values(1)


In [104]:
df_rest.columns = ["code_module", "code_presentation", "id_student", 0,1,2,3,4,5,6,7,8]

In [106]:
df_rest_final = df_rest.merge(df_clean_withdrawn, on=["code_module", "code_presentation", "id_student"])

In [108]:
df_rest_final.to_csv("oui.csv")